In [53]:
""" LIST SIP INBOUND TRUNKS """
from dotenv import load_dotenv 
from livekit import api
from livekit.protocol.sip import ListSIPInboundTrunkRequest

load_dotenv()

async def main():
  livekit_api = api.LiveKitAPI()

  rules = await livekit_api.sip.list_sip_inbound_trunk(
    ListSIPInboundTrunkRequest()
  )
  print(f"{rules}")

  await livekit_api.aclose()

(await main())

items {
  sip_trunk_id: "ST_2CrNutUodeAF"
  name: "trunk_+447723180506"
  numbers: "+447723180506"
}
items {
  sip_trunk_id: "ST_E4GjkXqaCyqr"
  name: "trunk_+13614281772"
  numbers: "+13614281772"
}



In [32]:
""" CREATE SIP INBOUND TRUNK """
from livekit import api 
from livekit.protocol.sip import CreateSIPInboundTrunkRequest, SIPInboundTrunkInfo

async def main(twilio_number: str):
    livekit_api = api.LiveKitAPI()

    trunk = SIPInboundTrunkInfo(
    name = f"trunk_{twilio_number}",
    numbers = [twilio_number],
    )
  
    request = CreateSIPInboundTrunkRequest(
        trunk = trunk
    )
    
    trunk = await livekit_api.sip.create_sip_inbound_trunk(request)
    
    await livekit_api.aclose()

twilio_number = "+447723180506"
await main(twilio_number)

In [30]:
""" DELETE SIP INBOUND TRUNK """
from livekit import api
from livekit.protocol.sip import DeleteSIPTrunkRequest 
async def delete_sip_trunk(trunk_id: str):
    livekit_api = api.LiveKitAPI()
    
    try:
        await livekit_api.sip.delete_sip_trunk(DeleteSIPTrunkRequest(sip_trunk_id=trunk_id))
        print(f"Successfully deleted SIP trunk with ID: {trunk_id}")
    except Exception as e:
        print(f"Error deleting SIP trunk: {str(e)}")
    finally:
        await livekit_api.aclose()

# Example usage:
trunk_id = "ST_WCmfQowTZB2J"  # Replace with actual trunk ID
await delete_sip_trunk(trunk_id)

Successfully deleted SIP trunk with ID: ST_WCmfQowTZB2J


In [54]:
""" CREATE SIP DISPATCH RULE """
from dotenv import load_dotenv 
from livekit import api 
from livekit.protocol.sip import (
    CreateSIPDispatchRuleRequest,
    SIPDispatchRule,
    SIPDispatchRuleIndividual
)

load_dotenv()

async def main(twilio_number: str, trunk_id: str):

  livekit_api = api.LiveKitAPI()

  # Create a dispatch rule to place each caller in a separate room
  rule = SIPDispatchRule(
    dispatch_rule_individual = SIPDispatchRuleIndividual(
      room_prefix = f'call-_{twilio_number}_',
      pin = ''
    )   
  )

  request = CreateSIPDispatchRuleRequest(
    rule = rule,
    name = f'rule_{twilio_number}_{trunk_id}',
    trunk_ids = [ 
      trunk_id,
    ],  
    hide_phone_number = False
  )

  try:
    dispatchRule = await livekit_api.sip.create_sip_dispatch_rule(request)
    print(f"Successfully created {dispatchRule}")
  except api.twirp_client.TwirpError as e:
    print(f"{e.code} error: {e.message}")

  await livekit_api.aclose()

twilio_number = "+13614281772"
trunk_id = "ST_E4GjkXqaCyqr"
(await main(twilio_number, trunk_id))

Successfully created sip_dispatch_rule_id: "SDR_9dzzQRh4ySsA"
rule {
  dispatch_rule_individual {
    room_prefix: "call-_+13614281772_"
  }
}
trunk_ids: "ST_E4GjkXqaCyqr"
name: "rule_+13614281772_ST_E4GjkXqaCyqr"



In [55]:
""" LIST SIP DISPATCH RULES """
from dotenv import load_dotenv
from livekit import api
from livekit.protocol.sip import ListSIPDispatchRuleRequest, DeleteSIPDispatchRuleRequest

load_dotenv()


async def main():
  livekit_api = api.LiveKitAPI()

  rules = await livekit_api.sip.list_sip_dispatch_rule(
    ListSIPDispatchRuleRequest()
  )
  print(f"{rules}")

  await livekit_api.aclose()

(await main())

items {
  sip_dispatch_rule_id: "SDR_9dzzQRh4ySsA"
  rule {
    dispatch_rule_individual {
      room_prefix: "call-_+13614281772_"
    }
  }
  trunk_ids: "ST_E4GjkXqaCyqr"
  name: "rule_+13614281772_ST_E4GjkXqaCyqr"
}
items {
  sip_dispatch_rule_id: "SDR_exhXDQ9DBwBv"
  rule {
    dispatch_rule_individual {
      room_prefix: "call-_+447723180506_"
    }
  }
  trunk_ids: "ST_2CrNutUodeAF"
  name: "rule_+447723180506_ST_2CrNutUodeAF"
}



In [49]:
""" DELETE SIP DISPATCH RULE """
from dotenv import load_dotenv
from livekit import api
from livekit.protocol.sip import ListSIPDispatchRuleRequest, DeleteSIPDispatchRuleRequest

load_dotenv()


async def main(dispatch_rule_id: str):
    livekit_api = api.LiveKitAPI()

    # First get all rules
    rules = await livekit_api.sip.list_sip_dispatch_rule(
        ListSIPDispatchRuleRequest()
    )
    print(f"Found rules: {rules}")

    await livekit_api.sip.delete_sip_dispatch_rule(
        DeleteSIPDispatchRuleRequest(
            sip_dispatch_rule_id=dispatch_rule_id
        )
    )
    print(f"Deleted rule with ID: {dispatch_rule_id}")

    await livekit_api.aclose()

(await main("SDR_9oiituE5jeyG"))

Found rules: items {
  sip_dispatch_rule_id: "SDR_9oiituE5jeyG"
  rule {
    dispatch_rule_individual {
      room_prefix: "call-_+447723180506-"
    }
  }
  trunk_ids: "ST_2CrNutUodeAF"
  name: "rule_+447723180506_ST_2CrNutUodeAF"
}

Deleted rule with ID: SDR_9oiituE5jeyG


In [ ]:
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List
from humanloop import Humanloop
from dotenv import load_dotenv
import os

""" URL TO MARKDOWN """

load_dotenv()

async def url_markdown(urls: List[str]):
    global scraped_content
    for url in urls:
        print("url:", url)
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        print("agent_name:", agent_name)
        
        # Map and scrape the URL
        mapped_url = await map_url(url)
        print("\n\nmapped_url:", mapped_url)
        scraped_content = await scrape_url(mapped_url[:100], user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        web_content = ""
        for i in scraped_content:
            web_content += i['content']
        
        # Save content to markdown file
        filename = f"{agent_name}.md"
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(web_content)
        
        print(f"Content saved to {filename}")
    
    return filename

# Example usage:
urls = [
    "https://www.perlonlabs.com/"
]

response = await url_markdown(urls)

print("response", response)

In [1]:
import uuid
import requests
from urllib.parse import urlparse
from services.knowledge_base.web_scrape import map_url, scrape_url
from services.chat.chat import llm_response
from typing import List, Dict
import openai
from humanloop import Humanloop
from dotenv import load_dotenv
import os

load_dotenv()

client = Humanloop(
    api_key=os.getenv("HUMANLOOP_API_KEY"),
)

async def create_opening_line(scraped_content: str) -> str:
    # Assuming you have OpenAI API setup
    user_prompt = f"""
    # Instructions:
    Based on this website content, create an engaging opening line for a customer service agent, like  <example_opening_line>"Welcome to WeCreate👋! Looking to boost your brand's online presence? What's your biggest goal right now?</example_opening_line>
    IMPORTANT: You must only respond with the opening line, no other text. No speech marks either, only the text.
    # Website content:
    {scraped_content[:1000]}"""
    
    system_prompt = "You are an expert at creating opening lines for customer service agents. You are given a website content and you are tasked with creating a friendly opening line for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agent_instructions(scraped_content: str) -> str:
    user_prompt = f"""
    # Instructions:
    Based on this website content, create a short sentence that begins with the name of the Company, and a short description saying what it does, like "WeCreate, a branding & digital marketing agency based Singapore, Hong Kong, and the UK."
    IMPORTANT: You must only respond with the instructions, no other text.
    # Website content:
    {scraped_content[:1000]}
    """
    
    system_prompt = "You are an expert at creating instructions for customer service agents. You are given a website content and you are tasked with creating instructions for a customer service agent."
    
    response = await llm_response(system_prompt, user_prompt)
    return response

async def create_agents_from_urls(urls: List[str]):
    global scraped_content
    responses = []
    for url in urls:
        print("url:",url)
        # Get domain name for agent name
        domain = urlparse(url).netloc
        agent_name = domain.replace("www.", "")
        print("agent_name:",agent_name)
        # Map and scrape the URL
        mapped_url = await map_url(url)
        print("\n\nmapped_url:",mapped_url)
        scraped_content = await scrape_url(mapped_url[:100], user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        web_content = ""
        for i in scraped_content:
            web_content += i['content']
        web_content = web_content[:10000]

        # Generate opening line from content
        opening_line = await create_opening_line(web_content)
        print("\n\nopening line:",opening_line)

        company_details = await create_agent_instructions(web_content)

        print("\n\ncompany details:",company_details)
        ## abstracted away lead form content, as that's determined by the user and stored in the metadata

        humanloop_prompt = client.prompts.get(
            id="pr_BiPy5GsclDI8uAzmHR8S7",
        )
        agent_instructions = humanloop_prompt.template[0]['content'].replace("{{company_details}}", company_details)

        # Create data source dict
        data_source = [{
            "id": str(uuid.uuid4()),
            "title": url,
            "data_type": "web"
        }]
        
        features = {
                    "lead_gen": {
                        "enabled": True
                    },
                    "callTransfer": {
                        "enabled": False
                    },
                    "notifyOnInterest": {
                        "enabled": False
                    },
                    "appointmentBooking": {
                        "enabled": False
                    }
                }
        
        form_fields = {
                "fields": [
                    {
                    "type": "text",
                    "label": "Full name"
                    },
                    {
                    "type": "email",
                    "label": "Email address"
                    }
                ]
            }
        
        """ POST to create new agent """
        # Prepare payload
        payload = {
            "agentName": agent_name,
            "agentPurpose": "text-chatbot-agent",
            "dataSource": data_source,
            "openingLine": opening_line,
            "language": "en-GB", 
            "voice": "Alex K",
            "voiceProvider": "cartesia",
            "instructions": agent_instructions,
            "features": features,
            "form_fields": form_fields,
            "company_name": agent_name
        }

        # POST request
        response = requests.post(
            "https://flowon.ai/api/v1/livekit/new_agent",
            json=payload,
            headers={"x-user-id": "user_2mmXezcGmjZCf88gT2v2waCBsXv"}
        )
        
        if response.status_code == 200:
            print(f"Successfully created agent for {url}")
            responses.append(response.json())  # Add response to list
        else:
            print(f"Failed to create agent for {url}: {response.text}")

# Example usage:
urls = [
"https://www.thesmallbizexpert.co.uk/"
]

response = await create_agents_from_urls(urls)
# agent_id = response['data'][0]['id']

# print("agent_id", agent_id)

url: https://cosmarketingagency.com/
agent_name: cosmarketingagency.com
hello, map_url


mapped_url: ['https://cosmarketingagency.com', 'https://cosmarketingagency.com/category/uncategorized', 'https://cosmarketingagency.com/category/marketing-content', 'https://cosmarketingagency.com/category/tips', 'https://cosmarketingagency.com/category/social-media', 'https://cosmarketingagency.com/category/tips-and-ideas', 'https://cosmarketingagency.com/category/video-marketing', 'https://cosmarketingagency.com/category/holiday-marketing', 'https://cosmarketingagency.com/category/search-engine-optimization', 'https://cosmarketingagency.com/category/analytics', 'https://cosmarketingagency.com/category/website-design', 'https://cosmarketingagency.com/category/advertising', 'https://cosmarketingagency.com/category/branding', 'https://cosmarketingagency.com/category/consulting', 'https://cosmarketingagency.com/category/photography', 'https://cosmarketingagency.com/category/about', 'https://cosmarket

[2025-01-10 18:36:23 - services.knowledge_base.web_scrape:147 - ERROR] HTTP error while scraping https://cosmarketingagency.com/digital-advertising-company: Unexpected error during scrape URL: Status code 429. Rate limit exceeded. Consumed (req/min): 21, Remaining (req/min): 0. Upgrade your plan at https://firecrawl.dev/pricing for increased rate limits or please retry after 1s, resets at Fri Jan 10 2025 18:36:23 GMT+0000 (Coordinated Universal Time) - No additional error details provided.
[2025-01-10 18:36:23 - services.knowledge_base.web_scrape:147 - ERROR] HTTP error while scraping https://cosmarketingagency.com/more-content-services: Unexpected error during scrape URL: Status code 429. Rate limit exceeded. Consumed (req/min): 22, Remaining (req/min): 0. Upgrade your plan at https://firecrawl.dev/pricing for increased rate limits or please retry after 1s, resets at Fri Jan 10 2025 18:36:23 GMT+0000 (Coordinated Universal Time) - No additional error details provided.
[2025-01-10 18:3

hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, sliding_window_chunking
hello, sliding_window_chunking
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding


[2025-01-10 18:36:30 - services.knowledge_base.web_scrape:147 - ERROR] HTTP error while scraping https://cosmarketingagency.com/brand-building-company: Unexpected error during scrape URL: Status code 429. Rate limit exceeded. Consumed (req/min): 23, Remaining (req/min): 0. Upgrade your plan at https://firecrawl.dev/pricing for increased rate limits or please retry after 54s, resets at Fri Jan 10 2025 18:37:23 GMT+0000 (Coordinated Universal Time) - No additional error details provided.
[2025-01-10 18:36:30 - services.knowledge_base.web_scrape:147 - ERROR] HTTP error while scraping https://cosmarketingagency.com/logo-design-company: Unexpected error during scrape URL: Status code 429. Rate limit exceeded. Consumed (req/min): 22, Remaining (req/min): 0. Upgrade your plan at https://firecrawl.dev/pricing for increased rate limits or please retry after 54s, resets at Fri Jan 10 2025 18:37:23 GMT+0000 (Coordinated Universal Time) - No additional error details provided.
[2025-01-10 18:36:30 

hello, sliding_window_chunking
hello, sliding_window_chunking
hello, sliding_window_chunking
hello, sliding_window_chunking
hello, sliding_window_chunking
hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding
hello, get_embedding


...

hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db


[2025-01-10 18:47:53 - services.knowledge_base.web_scrape:147 - ERROR] HTTP error while scraping https://ezedim.com/seo-services: Unexpected error during scrape URL: Status code 429. Rate limit exceeded. Consumed (req/min): 28, Remaining (req/min): 0. Upgrade your plan at https://firecrawl.dev/pricing for increased rate limits or please retry after 25s, resets at Fri Jan 10 2025 18:48:18 GMT+0000 (Coordinated Universal Time) - No additional error details provided.


hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db
hello, insert_to_db


[2025-01-10 18:48:00 - services.knowledge_base.web_scrape:147 - ERROR] HTTP error while scraping https://ezedim.com/gallery: Unexpected error during scrape URL: Status code 429. Rate limit exceeded. Consumed (req/min): 29, Remaining (req/min): 0. Upgrade your plan at https://firecrawl.dev/pricing for increased rate limits or please retry after 18s, resets at Fri Jan 10 2025 18:48:18 GMT+0000 (Coordinated Universal Time) - No additional error details provided.
[2025-01-10 18:48:00 - services.knowledge_base.web_scrape:203 - ERROR] Error processing site https://ezedim.com/gallery: RetryError[<Future at 0x110ada090 state=finished raised HTTPError>]
Traceback (most recent call last):
  File "/Users/michaelali/Repos/flowon_ai/venv/lib/python3.12/site-packages/tenacity/_asyncio.py", line 50, in __call__
    result = await fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michaelali/Repos/flowon_ai/backend/services/knowledge_base/web_scrape.py", line 143, in scrape_with

hello, sliding_window_chunking
hello, get_embedding
hello, get_embedding
hello, insert_to_db
hello, insert_to_db
end, scrape_url


opening line: Hi there! Welcome to Ezedim Digital Marketing. Ready to boost your website's search engine rankings and attract more visitors? How can we help optimize your online presence today?


company details: Ezedim Digital Marketing, a company specializing in Search Engine Optimization (SEO) strategies to improve website and social media rankings.
Successfully created agent for https://ezedim.com/


In [ ]:
import os
import re

def duplicate_files(folder_path, original_name, new_name, variable_name, new_value):
    """
    Duplicates HTML and JS files while modifying a specified variable.
    
    Args:
        folder_path (str): Path to the folder containing the files
        original_name (str): Base name of the original files (without extension)
        new_name (str): Base name for the new files (without extension)
        variable_name (str): Name of the variable to modify
        new_value: New value to set for the variable
    """
    
    # Handle both HTML and JS files
    for ext in ['.html', '.js']:
        original_file = os.path.join(folder_path, f"{original_name}{ext}")
        new_file = os.path.join(folder_path, f"{new_name}{ext}")
        
        # Check if original file exists
        if not os.path.exists(original_file):
            print(f"Original file {original_file} not found")
            continue
            
        try:
            # Read the original file
            with open(original_file, 'r', encoding='utf-8') as file:
                content = file.read()
            
            # Replace the variable value
            # This pattern matches variable declarations in both JS and HTML
            pattern = f'(const|let|var)?\s*{variable_name}\s*=\s*["\']?([^"\';]+)["\']?'
            new_content = re.sub(pattern, f'\g<1> {variable_name} = "{new_value}"', content)
            
            # Write to new file
            with open(new_file, 'w', encoding='utf-8') as file:
                file.write(new_content)
                
            print(f"Successfully created {new_file}")
            
        except Exception as e:
            print(f"Error processing {original_file}: {str(e)}")


# Example usage
for url in urls:
    duplicate_files('./my_folder', 'original', 'copy', 'apiKey', 'new-api-key-value')

In [ ]:
""" CHAT WITH AGENT"""
from services.chat.lk_chat import lk_chat_process

async def test_chat(message: str , agent_id: str):
    room_name = "test_room_45iii22ij6ii"

    try:
        # Get the response generator
        response_generator = lk_chat_process(message, agent_id, room_name)
        
        print("\n\nAgent response:\n\n")
        # Stream the responses
        async for chunk in response_generator:
            # In a real application, you might want to send this to a websocket or other stream
            print('\033[96m' + chunk + '\033[0m', end='', flush=True)   

    except Exception as e:
        print(f"Error occurred: {str(e)}")

await test_chat(
    message="payment message standards to be observed by all PSPs", 
    agent_id="a706a185-fe32-4331-b35a-d241d29e9154")

In [ ]:
from services.chat.chat import similarity_search

response = await similarity_search(
    query="tell me about the company", 
    data_source={'web': ['https://nogood.io'], 'text_files': []},
    user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")

response

### Recall Meeting bots

In [ ]:
import requests

# API endpoint
url = "https://us-west-2.recall.ai/api/v1/bot/"

# Your authorization token
token = "209a894215526cd61d70ed2f77df6cb39f6b0e4d"

# Request headers
headers = {
    "Authorization": f"Token {token}",
    "Content-Type": "application/json"
}

# Request payload
payload = {
    "meeting_url": "https://meet.google.com/stn-gobs-xcr",
    "bot_name": "Bot"
}

# Send POST request
response = requests.post(url, headers=headers, json=payload)

# Check response
print(response.status_code)
print(response.json())

In [ ]:
bot_joining = response.json()

bot_joining

In [ ]:
import requests

url = "https://us-west-2.recall.ai/api/v1/bot/e09e55e2-a067-473d-80be-18b9725ba7b8/"

headers = {
    "accept": "application/json",
    "Authorization": "209a894215526cd61d70ed2f77df6cb39f6b0e4d"
}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
response.json()